In [1]:
import datetime

import pandas as pd
from datetime import datetime

# Load raw medication data from Round
round_raw = pd.read_csv('data/Round Health 2021-11-27-21-34.csv')

# Clean round_raw
round_wrk = round_raw.copy()
# Remove irrelevant columns
round_wrk = round_wrk.drop(columns=['Start time', 'End time', 'Quantity', 'Timezone', 'Date'])

# Standardize strength values
round_wrk.Strength = round_wrk.Strength.replace('Lamotrigine', '', regex=True)
round_wrk.Strength = round_wrk.Strength.replace('mg', '', regex=True)
round_wrk.Strength = [float(x) for x in round_wrk.Strength]

# Convert dates from str to date
round_wrk['Taken On'] = [datetime.strptime(x, '%Y-%m-%d') if x != '--' else datetime.strptime('0001-01-01', '%Y-%m-%d') for x in round_wrk['Taken On']]
round_wrk['Taken'] = [0 if x == '--' else 1 for x in round_wrk['Taken At']]
round_wrk['Taken At'] = [datetime.strptime(x, '%I:%M %p').time() if x != '--' else datetime.strptime('01:00 AM', '%I:%M %p').time() for x in round_wrk['Taken At']]
round_wrk

,Name,Strength,Taken On,Taken At,Taken
0,Zoloft,25.0,2020-07-20 00:00:00,09:44:00,1
1,Zoloft,25.0,2020-07-21 00:00:00,08:47:00,1
2,Zoloft,25.0,2020-07-22 00:00:00,08:58:00,1
3,Zoloft,25.0,2020-07-23 00:00:00,08:33:00,1
4,Zoloft,25.0,2020-07-24 00:00:00,08:50:00,1
...,...,...,...,...,...
548,Lamictal,100.0,2021-11-25 00:00:00,07:59:00,1
549,Zoloft,50.0,2021-11-26 00:00:00,10:37:00,1
550,Lamictal,100.0,2021-11-26 00:00:00,10:37:00,1
551,Zoloft,50.0,2021-11-27 00:00:00,10:47:00,1


In [2]:
# Load mood data from Pixels
pixels_raw = pd.read_json('data/PixelsExport_2021-11-27.json')

# Remove columns
pixels_wrk = pixels_raw.copy()
pixels_wrk = pixels_wrk.drop(columns=['notes'])
pixels_wrk['date'] = [str(x) for x in pixels_wrk.date]
pixels_wrk

,mood,emotions,date
0,4,"[chill, stress, excitement, nerves, anger]",2021-01-03 00:00:00
1,3,"[tiredness, love, nerves, chill, optimism, fear]",2021-01-04 00:00:00
2,3,"[joy, emptiness, chill, excitement, tiredness]",2021-01-05 00:00:00
3,2,"[emptiness, tiredness, stress, chill, happines...",2021-01-06 00:00:00
4,1,"[remorse, excitement, tiredness, nerves, stres...",2021-01-07 00:00:00
...,...,...,...
323,4,[],2021-11-22 00:00:00
324,2,[],2021-11-23 00:00:00
325,3,[],2021-11-24 00:00:00
326,3,[],2021-11-25 00:00:00


In [262]:
import numpy as np

# Combine tables
round_wrk['date'] = [str(x) for x in round_wrk['Taken On']]
master = pixels_wrk.merge(round_wrk, how='left', on='date')
master.Taken[master.Taken != 1.0] = [0]
master = master.drop(columns=['Taken On'])
master['Taken At'] = [x if type(x) is not float else '00:00:00' for x in master['Taken At']]
master

/var/folders/zl/f1r1src11s71p47cjq22tpbw0000gn/T/ipykernel_28149/2048732795.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,mood,emotions,date,Name,Strength,Taken At,Taken
0,4,"[chill, stress, excitement, nerves, anger]",2021-01-03 00:00:00,Zoloft,50.0,11:33:00,1.0
1,3,"[tiredness, love, nerves, chill, optimism, fear]",2021-01-04 00:00:00,Zoloft,50.0,12:02:00,1.0
2,3,"[joy, emptiness, chill, excitement, tiredness]",2021-01-05 00:00:00,Zoloft,50.0,12:07:00,1.0
3,2,"[emptiness, tiredness, stress, chill, happines...",2021-01-06 00:00:00,Zoloft,50.0,11:18:00,1.0
4,1,"[remorse, excitement, tiredness, nerves, stres...",2021-01-07 00:00:00,Zoloft,50.0,11:06:00,1.0
...,...,...,...,...,...,...,...
327,3,[],2021-10-22 00:00:00,Lamictal,50.0,09:48:00,1.0
328,4,[],2021-10-23 00:00:00,Zoloft,50.0,10:35:00,1.0
329,4,[],2021-10-23 00:00:00,Lamictal,50.0,10:35:00,1.0
330,4,[],2021-10-24 00:00:00,Zoloft,50.0,11:16:00,1.0


In [263]:
master.describe()

,mood,Strength,Taken
count,332.000000,316.000000,332.000000
mean,3.096386,48.892405,0.951807
std,0.950299,5.152389,0.214497
min,1.000000,25.000000,0.000000
25%,3.000000,50.000000,1.000000
50%,3.000000,50.000000,1.000000
75%,4.000000,50.000000,1.000000
max,5.000000,50.000000,1.000000


In [265]:
# Create additional time data

master['dt_val'] = [pd.to_datetime(x) for x in master.date]
master['week'] = [x.isocalendar()[1] for x in master.dt_val]
master['month'] = [x.month for x in master.dt_val]
master['day_of_week_num'] = [x.isocalendar()[2] for x in master.dt_val]

weekday_dict = {'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7}
weekday_dict_corrected = dict((v,k) for k,v in weekday_dict.items())


def get_weekday(weekday_num):
    return weekday_dict_corrected[weekday_num]


master['day_of_week'] = [get_weekday(x) for x in master['day_of_week_num']]

def day(date):
    return date.day_of_year


def get_season(date):
    if day(date) < 79:
        return 'Winter'
    elif 79 <= day(date) < 172:
        return 'Spring'
    elif 172 <= day(date) < 265:
        return 'Summer'
    elif 265 <= day(date) < 355:
        return 'Fall'
    else:
        return 'Winter'


master['season'] = [get_season(x) for x in master.dt_val]
master = master.drop(columns=['day_of_week_num', 'date'])

master['Name'] = [x if type(x) is str else 'Not Taken' for x in master.Name]

In [266]:
def time_to_numeric(t):
    ts = str(t)
    (h, m, s) = ts.split(':')
    result = int(h) * 3600 + int(m) * 60 + int(s)
    return result

master['TimeNumeric'] =[time_to_numeric(x) for x in master['Taken At']]
master

,mood,emotions,Name,Strength,Taken At,Taken,dt_val,week,month,day_of_week,season,TimeNumeric
0,4,"[chill, stress, excitement, nerves, anger]",Zoloft,50.0,11:33:00,1.0,2021-01-03,53,1,Saturday,Winter,41580
1,3,"[tiredness, love, nerves, chill, optimism, fear]",Zoloft,50.0,12:02:00,1.0,2021-01-04,1,1,Sunday,Winter,43320
2,3,"[joy, emptiness, chill, excitement, tiredness]",Zoloft,50.0,12:07:00,1.0,2021-01-05,1,1,Monday,Winter,43620
3,2,"[emptiness, tiredness, stress, chill, happines...",Zoloft,50.0,11:18:00,1.0,2021-01-06,1,1,Tuesday,Winter,40680
4,1,"[remorse, excitement, tiredness, nerves, stres...",Zoloft,50.0,11:06:00,1.0,2021-01-07,1,1,Wednesday,Winter,39960
...,...,...,...,...,...,...,...,...,...,...,...,...
327,3,[],Lamictal,50.0,09:48:00,1.0,2021-10-22,42,10,Thursday,Fall,35280
328,4,[],Zoloft,50.0,10:35:00,1.0,2021-10-23,42,10,Friday,Fall,38100
329,4,[],Lamictal,50.0,10:35:00,1.0,2021-10-23,42,10,Friday,Fall,38100
330,4,[],Zoloft,50.0,11:16:00,1.0,2021-10-24,42,10,Saturday,Fall,40560


In [251]:
# Create variance table
mood_weekly = master.groupby('week')['mood'].mean().reset_index()
mood_weekly['avg'] = mood_weekly.mood.copy()
mood_weekly = mood_weekly.drop(columns=['mood'])

In [253]:
mood_weekly['variance'] = master.groupby('week')['mood'].var().reset_index().mood
mood_weekly['habit_score'] = master.groupby('week')['Taken'].mean().reset_index().Taken
mood_weekly['time_variance'] = master.groupby('week')['TimeNumeric'].var().reset_index().TimeNumeric/10000000
mood_weekly

,week,avg,variance,habit_score,time_variance
0,1,2.857143,1.142857,1.000000,1.713051
1,2,3.000000,1.666667,1.000000,1.909680
2,3,2.571429,0.952381,0.857143,18.936206
3,4,2.857143,1.142857,0.857143,21.498051
4,5,2.857143,0.809524,1.000000,2.696451
5,6,2.714286,1.571429,1.000000,3.631766
6,7,3.000000,0.444444,1.000000,1.641300
7,8,3.142857,0.809524,1.000000,1.798731
8,9,3.714286,0.571429,1.000000,1.881566
9,10,3.000000,1.333333,1.000000,3.114480


In [267]:
mood_monthly = master.groupby('month')['mood'].mean().reset_index()
mood_monthly['avg'] = mood_monthly.mood.copy()
mood_monthly = mood_monthly.drop(columns=['mood'])
mood_monthly['variance'] = master.groupby('month')['mood'].var().reset_index().mood
mood_monthly['habit_score'] = master.groupby('month')['Taken'].mean().reset_index().Taken
mood_monthly['time_variance'] = master.groupby('month')['TimeNumeric'].var().reset_index().TimeNumeric/10000000
mood_monthly

,month,avg,variance,habit_score,time_variance
0,1,2.862069,1.123153,0.931034,12.223398
1,2,2.935484,0.795699,1.000000,2.293202
2,3,3.000000,1.200000,0.967742,5.470017
3,4,3.031250,0.740927,0.875000,17.587042
4,5,3.142857,0.831933,0.914286,18.922099
5,6,3.212121,0.922348,0.909091,16.513566
6,7,3.290323,0.612903,0.967742,6.959920
7,8,3.181818,0.965909,0.969697,9.492813
8,9,3.166667,0.557471,0.966667,7.274680
9,10,3.106383,1.227567,1.000000,1.383900


In [194]:
import plotly.express as px

varianceXweek = px.scatter(mood_weekly, x='week', y='variance', trendline='ols')
varianceXweek.show()

In [185]:
box = px.box(master, x='season', y='mood', color='Name')
box.show()

In [216]:
import researchpy as rp
import scipy.stats as stats

rp.ttest(group1 = master.mood[master.season == 'Summer'], group1_name='Summer', group2 = master.mood[master.season == 'Winter'], group2_name='Winter')

(   Variable      N      Mean        SD        SE  95% Conf.  Interval
 0    Summer   95.0  3.263158  0.853161  0.087532   3.089360  3.436956
 1    Winter   79.0  2.949367  0.972688  0.109436   2.731497  3.167237
 2  combined  174.0  3.120690  0.920120  0.069754   2.983011  3.258368,
                 Independent t-test   results
 0  Difference (Summer - Winter) =     0.3138
 1            Degrees of freedom =   172.0000
 2                             t =     2.2664
 3         Two side test p value =     0.0247
 4        Difference < 0 p value =     0.9877
 5        Difference > 0 p value =     0.0123
 6                     Cohen's d =     0.3451
 7                     Hedge's g =     0.3436
 8                 Glass's delta =     0.3678
 9                   Pearson's r =     0.1703)

In [215]:
rp.ttest(group1 = master.mood[master.Taken == 1.0], group1_name='Taken', group2 = master.mood[master.Taken == 0], group2_name='Not Taken')

(    Variable      N      Mean        SD        SE  95% Conf.  Interval
 0      Taken  316.0  3.091772  0.946738  0.053258   2.986985  3.196559
 1  Not Taken   16.0  3.187500  1.046821  0.261705   2.629689  3.745311
 2   combined  332.0  3.096386  0.950299  0.052154   2.993790  3.198982,
                   Independent t-test   results
 0  Difference (Taken - Not Taken) =    -0.0957
 1              Degrees of freedom =   330.0000
 2                               t =    -0.3926
 3           Two side test p value =     0.6949
 4          Difference < 0 p value =     0.3474
 5          Difference > 0 p value =     0.6526
 6                       Cohen's d =    -0.1006
 7                       Hedge's g =    -0.1004
 8                   Glass's delta =    -0.1011
 9                     Pearson's r =     0.0216)

In [220]:
varXmood = px.scatter(mood_weekly, x='avg', y='variance', trendline='ols')
varXmood.show()

In [222]:
varXmood_dens = px.density_heatmap(mood_weekly, x='avg', y='variance')
varXmood_dens.show()

In [225]:
varianceXhs = px.scatter(mood_weekly, x='habit_score', y='variance', trendline='ols')
varianceXhs.show()

In [227]:
averageXhs = px.scatter(mood_weekly, x='habit_score', y='avg', trendline='ols')
averageXhs.show()

In [257]:
varianceXtimevariance = px.scatter(mood_weekly, x='time_variance', y='avg', trendline='ols')
varianceXtimevariance.show()

In [270]:
fig = px.scatter(mood_monthly, x='month', y='variance', trendline='ols')
fig.show()

In [273]:
fig1 = px.scatter(mood_monthly, x='month', y='avg', trendline='ols')
fig1.show()

In [275]:
fig2 = px.scatter(mood_monthly, x='avg', y='variance')
fig2.show()

In [279]:
fig3 = px.scatter(mood_monthly, x='habit_score', y='variance', trendline='ols')
fig3.show()

In [284]:
fig4 = px.scatter(mood_monthly, x='time_variance', y='variance', trendline='ols')
fig4.show()

In [289]:
fig5 = px.scatter(mood_monthly, x='month', y='habit_score', trendline='ols')
fig5.show()